In [1]:
import re, pickle, math, time, os, copy
import train_utils, function_utils, dict_utils, build_feature_utils

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, ElasticNetCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


### Remove and fix some data

In [2]:
batch1 = pickle.load(open(os.path.join('.','Data','batch1.pkl'), 'rb'))
#remove batteries that do not reach 80% capacity
del batch1['b1c8']
del batch1['b1c10']
del batch1['b1c12']
del batch1['b1c13']
del batch1['b1c22']

numBat1 = len(batch1.keys())

batch2 = pickle.load(open(os.path.join('.','Data','batch2.pkl'),'rb'))
# There are four cells from batch1 that carried into batch2, we'll remove the data from batch2
# and put it with the correct cell from batch1
batch2_keys = ['b2c7', 'b2c8', 'b2c9', 'b2c15', 'b2c16']
batch1_keys = ['b1c0', 'b1c1', 'b1c2', 'b1c3', 'b1c4']
add_len = [662, 981, 1060, 208, 482];

for i, bk in enumerate(batch1_keys):
    batch1[bk]['cycle_life'] = batch1[bk]['cycle_life'] + add_len[i]
    for j in batch1[bk]['summary'].keys():
        if j == 'cycle':
            batch1[bk]['summary'][j] = np.hstack((batch1[bk]['summary'][j], batch2[batch2_keys[i]]['summary'][j] + len(batch1[bk]['summary'][j])))
        else:
            batch1[bk]['summary'][j] = np.hstack((batch1[bk]['summary'][j], batch2[batch2_keys[i]]['summary'][j]))
    last_cycle = len(batch1[bk]['cycles'].keys())
    for j, jk in enumerate(batch2[batch2_keys[i]]['cycles'].keys()):
        batch1[bk]['cycles'][str(last_cycle + j)] = batch2[batch2_keys[i]]['cycles'][jk]
        
## THIS IS MY CHANGE, 3/6/2021: b2c1 is an anomaly; 148 cycle life. REMOVE at cell above and below

del batch2['b2c7']
del batch2['b2c8']
del batch2['b2c9']
del batch2['b2c15']
del batch2['b2c16']

numBat2 = len(batch2.keys())

batch3 = pickle.load(open(os.path.join('.','Data','batch3.pkl'),'rb'))
# remove noisy channels from batch3
del batch3['b3c37']
del batch3['b3c2']
del batch3['b3c23']
del batch3['b3c32']
del batch3['b3c38']
del batch3['b3c39']

numBat3 = len(batch3.keys())

numBat = numBat1 + numBat2 + numBat3
bat_dict = {'LFP_data': {**batch1, **batch2, **batch3}}

In [3]:
# index used in the nature paper
# test_ind = np.hstack((np.arange(0,(numBat1+numBat2),2),83))
test_ind = np.arange(0,(numBat1+numBat2),2)
train_ind = np.arange(1,(numBat1+numBat2-1),2)
secondary_test_ind = np.arange(numBat-numBat3,numBat);

# # remove the anomalous cell, b2c1
test_ind = np.delete(test_ind, 21)
test_ind = np.append(test_ind, numBat1+numBat2-1)

In [4]:
bat_index_dict = dict_utils.create_bat_index_dict(bat_dict['LFP_data'].keys(), bat_dict['LFP_data'])
bat_dict = function_utils.set_stepIndex(bat_dict, bat_index_dict)

# Check for error in the step index assignment
for cellNo in bat_dict['LFP_data'].keys():
    for cycleNo in list(bat_dict['LFP_data'][cellNo]['cycles'].keys())[1:202]:
        checkCell = bat_dict['LFP_data'][cellNo]['cycles'][cycleNo]
        idxCh = np.where(checkCell['Step_index'] == 3)[0]
        idxDis = np.where(checkCell['Step_index'] == 5)[0]

        if np.mean(checkCell['Qd'][idxCh][-10:]) > 0.1: print("The mean Qd for charging index is higher than 0.1 for", cellNo, cycleNo)
        if checkCell['Qd'][idxDis[0]] > 0.1: print("The starting Qd is higher than 0.1 for", cellNo, cycleNo)

# Manual fix for certain cell/cycle
fixCell = bat_dict['LFP_data']['b2c27']['cycles']
for keyName in fixCell['118'].keys():
    fixCell['118'][keyName] = fixCell['117'][keyName]

Rest index is empty for b1c45 270
The mean Qd for charging index is higher than 0.1 for b2c12 252
The starting Qd is higher than 0.03 for b2c12 252
The starting Qd is higher than 0.03 for b2c27 118
The starting Qd is higher than 0.03 for b2c44 247
Rest index is empty for b2c44 247
The starting Qd is higher than 0.1 for b2c27 118


## 1. Only considers CC region and interpolate

In [5]:
stepTotal = 100
testDict = copy.deepcopy(bat_dict)

for cellNo in testDict['LFP_data'].keys():
    for cycleNo in testDict['LFP_data'][cellNo]['cycles'].keys():
        if cycleNo == '0': continue
        testCellwCycle = testDict['LFP_data'][cellNo]['cycles'][cycleNo]
        
        idxDis = np.where(testCellwCycle['Step_index'] == 5)[0]
        idxCh = np.where(testCellwCycle['Step_index'] == 3)[0]
        # idxChCC = np.where(testCellwCycle['Step_index'] == 3)[0][0: np.where(testCellwCycle['Step_index'] == 4)[0][0]]

        linDictDis = function_utils.interpolate_data(xLimitList = [3.55, 2.05], step = stepTotal, cellDictOri = testCellwCycle, 
                                                 yKeyList = ['Qd', 'T_cell', 'V'], xKey = 'V', idxList = idxDis)
        for keyName in linDictDis.keys(): testCellwCycle[keyName] = linDictDis[keyName].copy()
        
        
        linDictCh = function_utils.interpolate_data(xLimitList = [0, 0.88], step = stepTotal, cellDictOri = testCellwCycle, 
                                                  yKeyList = ['Qc', 'T_cell'], xKey = 'Qc', idxList = idxCh)
        for keyName in linDictCh.keys(): testCellwCycle[keyName] = linDictCh[keyName].copy()

## 2. With both CC + CV

In [6]:
stepTotal = 100
testDictwCV = copy.deepcopy(bat_dict)

for cellNo in testDictwCV['LFP_data'].keys():
    for cycleNo in testDictwCV['LFP_data'][cellNo]['cycles'].keys():
        if cycleNo == '0': continue
        testCellwCycle = testDictwCV['LFP_data'][cellNo]['cycles'][cycleNo]
        
        idxDis = np.where(testCellwCycle['Step_index'] == 5)[0]
        idxCh = np.where(testCellwCycle['Step_index'] == 3)[0]
        # idxChCC = np.where(testCellwCycle['Step_index'] == 3)[0][0: np.where(testCellwCycle['Step_index'] == 4)[0][0]]

        linDictDis = function_utils.interpolate_data(xLimitList = [0, np.max(testCellwCycle['Qd'])], step = stepTotal, cellDictOri = testCellwCycle, 
                                                 yKeyList = ['Qd', 'T_cell'], xKey = 'Qd', idxList = idxDis)
        for keyName in linDictDis.keys(): testCellwCycle[keyName] = linDictDis[keyName].copy()
        
        
        linDictCh = function_utils.interpolate_data(xLimitList = [0, np.max(testCellwCycle['Qc'])], step = stepTotal, cellDictOri = testCellwCycle, 
                                                  yKeyList = ['Qc', 'T_cell'], xKey = 'Qc', idxList = idxCh)
        for keyName in linDictCh.keys(): testCellwCycle[keyName] = linDictCh[keyName].copy()

In [7]:
cycleList = [str(x) for x in np.arange(1,150,1)]
T_Q =  build_feature_utils.create_T_Q_lin_feature(testDict, cycleList)
dTdQ =  build_feature_utils.create_dTdQ_lin_feature(testDict, cycleList)

nature_dict = build_feature_utils.create_nature_feature_v2(bat_index_dict, testDict)
nature_dict_2to10 = build_feature_utils.create_nature_feature_v2_2to10(bat_index_dict, testDict)

TQKeynameUnique = function_utils.get_unique_keyname_tuple(T_Q)
dTdQKeynameUnique = function_utils.get_unique_keyname_tuple(dTdQ)
natureKeynameUnique = list(nature_dict.keys())[5:]
nature2to10KeynameUnique = list(nature_dict_2to10.keys())[5:]

In [8]:
# Create an overall battery dataframe with charging policy, cycle life, and average charge time
chargepolicylst = []
cyclelifelst = []
avgchargepolicylst = []
diffchargepolicylst = []
avgchargetimelst = []

timeCycles = list(map(str, np.arange(1,11,1))) #first 10 cycles for average charge time

for batno in testDict['LFP_data'].keys():
    chargepolicylst.append((testDict['LFP_data'][batno]['charge_policy']))
    cyclelifelst.append(int(testDict['LFP_data'][batno]['cycle_life']))
    
    totTime = 0
    for cycle in timeCycles:
        stopIdx = bat_index_dict[batno][cycle]['rest'][0]
        totTime += testDict['LFP_data'][batno]['cycles'][cycle]['t'][stopIdx]
    avgchargetimelst.append(totTime/len(timeCycles))

avgchargepolicylst = []
for policy in chargepolicylst:
    avgchargelst = re.findall(r'[0-9.]+', policy)
    avgchargelst = list(map(float, avgchargelst))
    a0 = avgchargelst[0]; a1 = avgchargelst[1]; a2 = avgchargelst[2]
    
    avgcharge = ((a0 * a1 / 100) + (a2 * (0.80 - a1 / 100))) / 0.80
    avgchargepolicylst.append(avgcharge)
    
    diffchargepolicy = a0 - a2
    diffchargepolicylst.append(diffchargepolicy)
       
bat_df = pd.DataFrame({'Bat code': pd.Series(list(testDict['LFP_data'].keys())),
                       'Charging policy': pd.Series(chargepolicylst),
                       'Avg charging policy' : pd.Series(avgchargepolicylst),
                       'Diff in charging policy' : pd.Series(diffchargepolicylst),
                       'Cycle life': pd.Series(cyclelifelst),
                       'Avg charging time' : pd.Series(avgchargetimelst)
})


In [9]:
# Y values - cycle life
y_train = np.log10(bat_df['Cycle life'][train_ind])
y_test = np.log10(bat_df['Cycle life'][test_ind])
y_secondary_test = np.log10(bat_df['Cycle life'][secondary_test_ind])
y_all = np.concatenate((y_train, y_test, y_secondary_test))

printCoef = False
lenList = [len(train_ind), len(train_ind) + len(test_ind), len(train_ind) + len(test_ind) + len(secondary_test_ind)]

In [10]:
cycleList = ['Average_2to10']

testMAEArray, testRMSEArray, secTestMAEArray, secTestRMSEArray = [np.array([]) for _ in range(4)]

for setCycleNo in cycleList:
    ## Create the dataframe
    # Initiate the feature dataframe
    feat_df = pd.DataFrame(columns=['dictionary', 'keyName', 'valueArray', 'rScore'])

    # To change: modeList
    modeList = ['Charge', 'Discharge']

    feat_df = pd.DataFrame(columns=['dictionary', 'keyName', 'valueArray', 'rScore'])

    # Variables

    featureDict = T_Q.copy()
    featureKeynameUniqueList = TQKeynameUnique.copy()
    keyCycle = setCycleNo
    sample_size = featureDict[('Discharge', 'Max-Min', setCycleNo)].size

    ## END

    featureKeynameList = []

    # All cells
    X = np.empty((sample_size,0))
    y = featureDict['cycle_life'].ravel()


    # Append the feature KEYS to be inspected into featureKeynameList
    for keyMode in modeList:
        ## NEED TO CHANGE THE RANGE
        for keyStat in featureKeynameUniqueList[1][2:9]:
            featureKeynameList.append((keyMode, keyStat, keyCycle))
            
    # Append the FEATURE into X array

    for feat in featureKeynameList:
        # ALL CELLS
        X = np.append(X, featureDict[feat].reshape(-1,1), axis=1)
        
    # rScore of individual feature with cycle life
    rScore = train_utils.feature_target_score(X, y, featureKeynameList, printTitle='ALL CELL', toPrint=False)

    # Append to dataframe
    for i in range(len(featureKeynameList)):
        newRow = {'dictionary': 'T_Q', 'keyName': featureKeynameList[i], 'valueArray': X[:, i], 'rScore': rScore[i]}
        feat_df = feat_df.append(newRow, ignore_index=True)
                
    ##                                      2. dT/dQ

    # Variables

    featureDict = dTdQ.copy()
    featureKeynameUniqueList = dTdQKeynameUnique.copy()
    keyCycle = setCycleNo
    sample_size = featureDict[('Charge', 'Maximum', setCycleNo)].size

    ## END

    featureKeynameList = []

    # All cells
    X = np.empty((sample_size,0))
    y = featureDict['cycle_life'].ravel()

    # Append the feature KEYS to be inspected into featureKeynameList
    # For Charge and Discharge
    for keyMode in modeList:
        ## NEED TO CHANGE THE RANGE
        for keyStat in featureKeynameUniqueList[1][3:10]:
            featureKeynameList.append((keyMode, keyStat, keyCycle))
            
    # Append the FEATURE into X array

    for feat in featureKeynameList:
        # ALL CELLS
        X = np.append(X, featureDict[feat].reshape(-1,1), axis=1)
        
    rScore = train_utils.feature_target_score(X, y, featureKeynameList, printTitle='ALL CELL', toPrint=False)

    # Append to dataframe
    for i in range(len(featureKeynameList)):
        newRow = {'dictionary': 'dTdQ', 'keyName': featureKeynameList[i], 'valueArray': X[:, i], 'rScore': rScore[i]}
        feat_df = feat_df.append(newRow, ignore_index=True)
            
    ##                                      3. Severson's features: Cycle 2 to 10

    # Append the feature KEYS to be inspected into featureKeynameList
    featureDict = nature_dict_2to10.copy()
    featureKeynameList = nature2to10KeynameUnique[:-1].copy()
    sample_size = featureDict['Q_V_Variance'].size

    ## END

    # All cells
    X = np.empty((sample_size,0))
    y = featureDict['cycle_life'].ravel()
            
    # Append the FEATURE into X array

    for feat in featureKeynameList:
        # ALL CELLS
        X = np.append(X, featureDict[feat].reshape(-1,1), axis=1)
        
    # rScore of individual feature with cycle life
    rScore = train_utils.feature_target_score(X, y, featureKeynameList, printTitle='ALL CELL', toPrint=False)

    # Append to dataframe
    for i in range(len(featureKeynameList)):
        newRow = {'dictionary': 'nature_dict', 'keyName': featureKeynameList[i], 'valueArray': X[:, i], 'rScore': rScore[i]}
        feat_df = feat_df.append(newRow, ignore_index=True)

## Train and plot the performance of benchmark models

In [11]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from function_utils import root_mean_squared_percentage_error

### 1. Qd cycle 2

In [12]:
feat_df.loc[34, :]

dictionary                                          nature_dict
keyName                                                   Qd_c2
valueArray    [1.0706892, 1.0753012, 1.0799223, 1.0797229, 1...
rScore                                                -0.032721
Name: 34, dtype: object

In [13]:
featQd2_all = np.vstack(feat_df.loc[34, 'valueArray'])

featQd2_train  = featQd2_all[train_ind]
featQd2_test = featQd2_all[test_ind]
featQd2_secondary_test = featQd2_all[secondary_test_ind]

scaler = StandardScaler().fit(featQd2_train)
featQd2_train_scaled = scaler.transform(featQd2_train)
featQd2_test_scaled = scaler.transform(featQd2_test)
featQd2_secondary_test_scaled = scaler.transform(featQd2_secondary_test)
featQd2_all_scaled = np.concatenate((featQd2_train_scaled, featQd2_test_scaled, featQd2_secondary_test_scaled))

In [14]:
# Train linear regression
regLN = LinearRegression().fit(featQd2_train_scaled, y_train)

benchQd2All = 10**regLN.predict(featQd2_all_scaled)
benchQd2Train = 10**regLN.predict(featQd2_train_scaled)
benchQd2Test = 10**regLN.predict(featQd2_test_scaled)
benchQd2SecTest = 10**regLN.predict(featQd2_secondary_test_scaled)

In [15]:
print("Benchmark: Linear regression using Qd at cycle 2",
    "\nMAE_all: %.1f" % (mean_absolute_error(y_true = 10**y_all, y_pred = benchQd2All)),
    "\nMAE_train: %.1f" % (mean_absolute_error(y_true = 10**y_train, y_pred = benchQd2Train)),
    "\nMAE_test: %.1f" % (mean_absolute_error(y_true = 10**y_test, y_pred = benchQd2Test)),
    "\nMAE_2ndTest: %.1f" % (mean_absolute_error(y_true = 10**y_secondary_test, y_pred = benchQd2SecTest)),
    
    "\nMAPE_all: %.1f %%" % (mean_absolute_percentage_error(y_true = 10**y_all, y_pred = benchQd2All)* 100),
    "\nMAPE_train: %.1f %%" % (mean_absolute_percentage_error(y_true = 10**y_train, y_pred = benchQd2Train)* 100),
    "\nMAPE_test: %.1f %%" % (mean_absolute_percentage_error(y_true = 10**y_test, y_pred = benchQd2Test)* 100),
    "\nMAPE_2ndTest: %.1f %%" % (mean_absolute_percentage_error(y_true = 10**y_secondary_test, y_pred = benchQd2SecTest)* 100),
    
    "\nRMSE_all: %.1f" % (math.sqrt(mean_squared_error(y_true = 10**y_all, y_pred = benchQd2All))),
    "\nRMSE_train: %.1f" % (math.sqrt(mean_squared_error(y_true = 10**y_train, y_pred = benchQd2Train))),
    "\nRMSE_test: %.1f" % (math.sqrt(mean_squared_error(y_true = 10**y_test, y_pred = benchQd2Test))),
    "\nRMSE_2ndTest: %.1f" % (math.sqrt(mean_squared_error(y_true = 10**y_secondary_test, y_pred = benchQd2SecTest))),
    
    "\nRMSPE_all: %.1f %%" % (root_mean_squared_percentage_error(y_true = 10**y_all, y_pred = benchQd2All) * 100),
    "\nRMSPE_train: %.1f %%" % (root_mean_squared_percentage_error(y_true = 10**y_train, y_pred = benchQd2Train)* 100),
    "\nRMSPE_test: %.1f %%" % (root_mean_squared_percentage_error(y_true = 10**y_test, y_pred = benchQd2Test)* 100),
    "\nRMSPE_2ndTest: %.1f %%" % (root_mean_squared_percentage_error(y_true = 10**y_secondary_test, y_pred = benchQd2SecTest)* 100)
    )
print("r2-scores")
print("All data r2-score: %.3f" % (r2_score(y_true = 10**y_all, y_pred = benchQd2All)))
print("Train data r2-score: %.3f" % (r2_score(y_true = 10**y_train, y_pred = benchQd2Train)))
print("Test data r2-score: %.3f" % (r2_score(y_true = 10**y_test, y_pred = benchQd2Test)))
print("Secondary test data r2-score: %.3f" % (r2_score(y_true = 10**y_secondary_test, y_pred = benchQd2SecTest)))

Benchmark: Linear regression using Qd at cycle 2 
MAE_all: 299.3 
MAE_train: 192.3 
MAE_test: 229.3 
MAE_2ndTest: 482.4 
MAPE_all: 32.3 % 
MAPE_train: 26.4 % 
MAPE_test: 26.9 % 
MAPE_2ndTest: 44.1 % 
RMSE_all: 430.4 
RMSE_train: 315.4 
RMSE_test: 388.1 
RMSE_2ndTest: 556.5 
RMSPE_all: 37.0 % 
RMSPE_train: 32.0 % 
RMSPE_test: 31.9 % 
RMSPE_2ndTest: 45.8 %
r2-scores
All data r2-score: -0.368
Train data r2-score: 0.047
Test data r2-score: -0.011
Secondary test data r2-score: -2.809


### 2. Average cycle life (constant value) of train set

In [16]:
benchMean = np.mean(10**y_train)
benchMeanAll = np.full(np.shape(y_all), benchMean)
benchMeanTrain = np.full(np.shape(y_train), benchMean)
benchMeanTest = np.full(np.shape(y_test), benchMean)
benchMeanSecTest = np.full(np.shape(y_secondary_test), benchMean)

print("Benchmark: Overall data average cycle life (i.e. constant %d)" % (benchMean),
    "\nMAE_all: %.1f" % (mean_absolute_error(y_true = 10**y_all, y_pred = benchMeanAll)),
    "\nMAE_train: %.1f" % (mean_absolute_error(y_true = 10**y_train, y_pred = benchMeanTrain)),
    "\nMAE_test: %.1f" % (mean_absolute_error(y_true = 10**y_test, y_pred = benchMeanTest)),
    "\nMAE_2ndTest: %.1f" % (mean_absolute_error(y_true = 10**y_secondary_test, y_pred = benchMeanSecTest)),
    
    "\nMAPE_all: %.1f %%" % (mean_absolute_percentage_error(y_true = 10**y_all, y_pred = benchMeanAll)* 100),
    "\nMAPE_train: %.1f %%" % (mean_absolute_percentage_error(y_true = 10**y_train, y_pred = benchMeanTrain)* 100),
    "\nMAPE_test: %.1f %%" % (mean_absolute_percentage_error(y_true = 10**y_test, y_pred = benchMeanTest)* 100),
    "\nMAPE_2ndTest: %.1f %%" % (mean_absolute_percentage_error(y_true = 10**y_secondary_test, y_pred = benchMeanSecTest)* 100),
    
    "\nRMSE_all: %.1f" % (math.sqrt(mean_squared_error(y_true = 10**y_all, y_pred = benchMeanAll))),
    "\nRMSE_train: %.1f" % (math.sqrt(mean_squared_error(y_true = 10**y_train, y_pred = benchMeanTrain))),
    "\nRMSE_test: %.1f" % (math.sqrt(mean_squared_error(y_true = 10**y_test, y_pred = benchMeanTest))),
    "\nRMSE_2ndTest: %.1f" % (math.sqrt(mean_squared_error(y_true = 10**y_secondary_test, y_pred = benchMeanSecTest))),
    
    "\nRMSPE_all: %.1f %%" % (root_mean_squared_percentage_error(y_true = 10**y_all, y_pred = benchMeanAll) * 100),
    "\nRMSPE_train: %.1f %%" % (root_mean_squared_percentage_error(y_true = 10**y_train, y_pred = benchMeanTrain)* 100),
    "\nRMSPE_test: %.1f %%" % (root_mean_squared_percentage_error(y_true = 10**y_test, y_pred = benchMeanTest)* 100),
    "\nRMSPE_2ndTest: %.1f %%" % (root_mean_squared_percentage_error(y_true = 10**y_secondary_test, y_pred = benchMeanSecTest)* 100)
    )
print("r2-scores")
print("All data r2-score: %.3f" % (r2_score(y_true = 10**y_all, y_pred = benchMeanAll)))
print("Train data r2-score: %.3f" % (r2_score(y_true = 10**y_train, y_pred = benchMeanTrain)))
print("Test data r2-score: %.3f" % (r2_score(y_true = 10**y_test, y_pred = benchMeanTest)))
print("Secondary test data r2-score: %.3f" % (r2_score(y_true = 10**y_secondary_test, y_pred = benchMeanSecTest)))

Benchmark: Overall data average cycle life (i.e. constant 673) 
MAE_all: 273.9 
MAE_train: 221.3 
MAE_test: 247.9 
MAE_2ndTest: 355.1 
MAPE_all: 32.1 % 
MAPE_train: 33.5 % 
MAPE_test: 31.9 % 
MAPE_2ndTest: 30.9 % 
RMSE_all: 390.3 
RMSE_train: 323.1 
RMSE_test: 389.1 
RMSE_2ndTest: 450.0 
RMSPE_all: 37.4 % 
RMSPE_train: 40.0 % 
RMSPE_test: 37.6 % 
RMSPE_2ndTest: 34.2 %
r2-scores
All data r2-score: -0.125
Train data r2-score: 0.000
Test data r2-score: -0.016
Secondary test data r2-score: -1.491


### 3. Variance model (10 cycles) from Severson

In [17]:
feat_df.loc[28, :]

dictionary                                          nature_dict
keyName                                            Q_V_Variance
valueArray    [-5.056549977467647, -4.862693001478201, -4.89...
rScore                                                 -0.12448
Name: 28, dtype: object

In [18]:
featVarQ_all = np.vstack(feat_df.loc[28, 'valueArray'])

featVarQ_train  = featVarQ_all[train_ind]
featVarQ_test = featVarQ_all[test_ind]
featVarQ_secondary_test = featVarQ_all[secondary_test_ind]

scaler = StandardScaler().fit(featVarQ_train)
featVarQ_train_scaled = scaler.transform(featVarQ_train)
featVarQ_test_scaled = scaler.transform(featVarQ_test)
featVarQ_secondary_test_scaled = scaler.transform(featVarQ_secondary_test)
featVarQ_all_scaled = np.concatenate((featVarQ_train_scaled, featVarQ_test_scaled, featVarQ_secondary_test_scaled))

# Train
regEN = ElasticNetCV(l1_ratio=np.linspace(0.01,1,100), cv=4, random_state=20, max_iter=100000, n_jobs=3, tol=1e-2, verbose=0, selection='random').fit(featVarQ_train_scaled, y_train)
regLN = LinearRegression().fit(featVarQ_train_scaled, y_train)
model = regEN

benchVarQAll = 10**model.predict(featVarQ_all_scaled)
benchVarQTrain = 10**model.predict(featVarQ_train_scaled)
benchVarQTest = 10**model.predict(featVarQ_test_scaled)
benchVarQSecTest = 10**model.predict(featVarQ_secondary_test_scaled)


print("Benchmark: ElasticNet on Variance model",
    "\nMAE_all: %.2f" % (mean_absolute_error(y_true = 10**y_all, y_pred = benchVarQAll)),
    "\nMAE_train: %.2f" % (mean_absolute_error(y_true = 10**y_train, y_pred = benchVarQTrain)),
    "\nMAE_test: %.2f" % (mean_absolute_error(y_true = 10**y_test, y_pred = benchVarQTest)),
    "\nMAE_2ndTest: %.2f" % (mean_absolute_error(y_true = 10**y_secondary_test, y_pred = benchVarQSecTest)),
    
    "\nMAPE_all: %.1f %%" % (mean_absolute_percentage_error(y_true = 10**y_all, y_pred = benchVarQAll)* 100),
    "\nMAPE_train: %.1f %%" % (mean_absolute_percentage_error(y_true = 10**y_train, y_pred = benchVarQTrain)* 100),
    "\nMAPE_test: %.1f %%" % (mean_absolute_percentage_error(y_true = 10**y_test, y_pred = benchVarQTest)* 100),
    "\nMAPE_2ndTest: %.1f %%" % (mean_absolute_percentage_error(y_true = 10**y_secondary_test, y_pred = benchVarQSecTest)* 100),
    
    "\nRMSE_all: %.2f" % (math.sqrt(mean_squared_error(y_true = 10**y_all, y_pred = benchVarQAll))),
    "\nRMSE_train: %.2f" % (math.sqrt(mean_squared_error(y_true = 10**y_train, y_pred = benchVarQTrain))),
    "\nRMSE_test: %.2f" % (math.sqrt(mean_squared_error(y_true = 10**y_test, y_pred = benchVarQTest))),
    "\nRMSE_2ndTest: %.2f" % (math.sqrt(mean_squared_error(y_true = 10**y_secondary_test, y_pred = benchVarQSecTest))),
    
    "\nRMSPE_all: %.1f %%" % (root_mean_squared_percentage_error(y_true = 10**y_all, y_pred = benchVarQAll) * 100),
    "\nRMSPE_train: %.1f %%" % (root_mean_squared_percentage_error(y_true = 10**y_train, y_pred = benchVarQTrain)* 100),
    "\nRMSPE_test: %.1f %%" % (root_mean_squared_percentage_error(y_true = 10**y_test, y_pred = benchVarQTest)* 100),
    "\nRMSPE_2ndTest: %.1f %%" % (root_mean_squared_percentage_error(y_true = 10**y_secondary_test, y_pred = benchVarQSecTest)* 100)
    )
print("r2-scores")
print("All data r2-score: %.3f" % (r2_score(y_true = 10**y_all, y_pred = benchVarQAll)))
print("Train data r2-score: %.3f" % (r2_score(y_true = 10**y_train, y_pred = benchVarQTrain)))
print("Test data r2-score: %.3f" % (r2_score(y_true = 10**y_test, y_pred = benchVarQTest)))
print("Secondary test data r2-score: %.3f" % (r2_score(y_true = 10**y_secondary_test, y_pred = benchVarQSecTest)))

Benchmark: ElasticNet on Variance model 
MAE_all: 279.80 
MAE_train: 192.97 
MAE_test: 240.35 
MAE_2ndTest: 410.21 
MAPE_all: 30.6 % 
MAPE_train: 25.9 % 
MAPE_test: 29.4 % 
MAPE_2ndTest: 36.5 % 
RMSE_all: 408.92 
RMSE_train: 317.09 
RMSE_test: 396.75 
RMSE_2ndTest: 495.82 
RMSPE_all: 35.8 % 
RMSPE_train: 31.1 % 
RMSPE_test: 36.8 % 
RMSPE_2ndTest: 39.2 %
r2-scores
All data r2-score: -0.234
Train data r2-score: 0.037
Test data r2-score: -0.057
Secondary test data r2-score: -2.024


### 4. Discharge model (10 cycles) from Severson

In [19]:
feat_df.loc[[28, 30, 31, 32, 34, 36], :]

,dictionary,keyName,valueArray,rScore
28,nature_dict,Q_V_Variance,"[-5.056549977467647, -4.862693001478201, -4.89...",-0.124480
30,nature_dict,Q_V_Skewness,"[0.23150057987661102, 0.41262728200575827, 0.4...",0.140957
31,nature_dict,Q_V_Kurtosis,"[0.5932016750339252, 0.9598960651344697, 0.970...",0.163680
32,nature_dict,Q_V_Minimum,"[-3.9107582681559294, -3.15456636763735, -3.10...",-0.134361
34,nature_dict,Qd_c2,"[1.0706892, 1.0753012, 1.0799223, 1.0797229, 1...",-0.032721
36,nature_dict,Qd_max-c2,"[0.003848100000000132, 0.004475400000000018, 0...",-0.223854


In [20]:
featDis_all = np.vstack(feat_df.loc[[28, 30, 31, 32, 34, 36], 'valueArray']).transpose()

featDis_train  = featDis_all[train_ind]
featDis_test = featDis_all[test_ind]
featDis_secondary_test = featDis_all[secondary_test_ind]

scaler = StandardScaler().fit(featDis_train)
featDis_train_scaled = scaler.transform(featDis_train)
featDis_test_scaled = scaler.transform(featDis_test)
featDis_secondary_test_scaled = scaler.transform(featDis_secondary_test)
featDis_all_scaled = np.concatenate((featDis_train_scaled, featDis_test_scaled, featDis_secondary_test_scaled))

# Train
regEN = ElasticNetCV(l1_ratio=np.linspace(0.01,1,100), cv=4, random_state=20, max_iter=100000, n_jobs=3, tol=1e-2, verbose=0, selection='random').fit(featDis_train_scaled, y_train)
regLN = LinearRegression().fit(featDis_train_scaled, y_train)
model = regEN

benchDisAll = 10**model.predict(featDis_all_scaled)
benchDisTrain = 10**model.predict(featDis_train_scaled)
benchDisTest = 10**model.predict(featDis_test_scaled)
benchDisSecTest = 10**model.predict(featDis_secondary_test_scaled)


print("Benchmark: ElasticNet on Discharge model",
    "\nMAE_all: %.2f" % (mean_absolute_error(y_true = 10**y_all, y_pred = benchDisAll)),
    "\nMAE_train: %.2f" % (mean_absolute_error(y_true = 10**y_train, y_pred = benchDisTrain)),
    "\nMAE_test: %.2f" % (mean_absolute_error(y_true = 10**y_test, y_pred = benchDisTest)),
    "\nMAE_2ndTest: %.2f" % (mean_absolute_error(y_true = 10**y_secondary_test, y_pred = benchDisSecTest)),
    
    "\nMAPE_all: %.1f %%" % (mean_absolute_percentage_error(y_true = 10**y_all, y_pred = benchDisAll)* 100),
    "\nMAPE_train: %.1f %%" % (mean_absolute_percentage_error(y_true = 10**y_train, y_pred = benchDisTrain)* 100),
    "\nMAPE_test: %.1f %%" % (mean_absolute_percentage_error(y_true = 10**y_test, y_pred = benchDisTest)* 100),
    "\nMAPE_2ndTest: %.1f %%" % (mean_absolute_percentage_error(y_true = 10**y_secondary_test, y_pred = benchDisSecTest)* 100),
    
    "\nRMSE_all: %.2f" % (math.sqrt(mean_squared_error(y_true = 10**y_all, y_pred = benchDisAll))),
    "\nRMSE_train: %.2f" % (math.sqrt(mean_squared_error(y_true = 10**y_train, y_pred = benchDisTrain))),
    "\nRMSE_test: %.2f" % (math.sqrt(mean_squared_error(y_true = 10**y_test, y_pred = benchDisTest))),
    "\nRMSE_2ndTest: %.2f" % (math.sqrt(mean_squared_error(y_true = 10**y_secondary_test, y_pred = benchDisSecTest))),
    
    "\nRMSPE_all: %.1f %%" % (root_mean_squared_percentage_error(y_true = 10**y_all, y_pred = benchDisAll) * 100),
    "\nRMSPE_train: %.1f %%" % (root_mean_squared_percentage_error(y_true = 10**y_train, y_pred = benchDisTrain)* 100),
    "\nRMSPE_test: %.1f %%" % (root_mean_squared_percentage_error(y_true = 10**y_test, y_pred = benchDisTest)* 100),
    "\nRMSPE_2ndTest: %.1f %%" % (root_mean_squared_percentage_error(y_true = 10**y_secondary_test, y_pred = benchDisSecTest)* 100)
    )
print("r2-scores")
print("All data r2-score: %.3f" % (r2_score(y_true = 10**y_all, y_pred = benchDisAll)))
print("Train data r2-score: %.3f" % (r2_score(y_true = 10**y_train, y_pred = benchDisTrain)))
print("Test data r2-score: %.3f" % (r2_score(y_true = 10**y_test, y_pred = benchDisTest)))
print("Secondary test data r2-score: %.3f" % (r2_score(y_true = 10**y_secondary_test, y_pred = benchDisSecTest)))

Benchmark: ElasticNet on Discharge model 
MAE_all: 294.05 
MAE_train: 182.04 
MAE_test: 239.42 
MAE_2ndTest: 466.24 
MAPE_all: 31.8 % 
MAPE_train: 24.4 % 
MAPE_test: 29.0 % 
MAPE_2ndTest: 42.3 % 
RMSE_all: 426.60 
RMSE_train: 306.41 
RMSE_test: 400.59 
RMSE_2ndTest: 543.03 
RMSPE_all: 36.9 % 
RMSPE_train: 29.3 % 
RMSPE_test: 35.8 % 
RMSPE_2ndTest: 44.2 %
r2-scores
All data r2-score: -0.343
Train data r2-score: 0.101
Test data r2-score: -0.077
Secondary test data r2-score: -2.628


### 5. Full model (10 cycles) from Severson

In [21]:
feat_df.loc[[28, 32, 34, 37, 38, 41, 43, 44, 47], :]

,dictionary,keyName,valueArray,rScore
28,nature_dict,Q_V_Variance,"[-5.056549977467647, -4.862693001478201, -4.89...",-0.124480
32,nature_dict,Q_V_Minimum,"[-3.9107582681559294, -3.15456636763735, -3.10...",-0.134361
34,nature_dict,Qd_c2,"[1.0706892, 1.0753012, 1.0799223, 1.0797229, 1...",-0.032721
37,nature_dict,Slope_2to10,"[0.00046190666666667455, 0.000530258333333337,...",0.000843
38,nature_dict,Intercept_2to10,"[1.0704779599999998, 1.074998661111111, 1.0800...",-0.049715
41,nature_dict,Avg_charge_time,"[13.374893999999992, 13.409150000000002, 13.35...",0.601873
43,nature_dict,Min_IR,"[0.016576905, 0.016839281, 0.016659655, 0.0162...",-0.380847
44,nature_dict,IR_cycle10-2,"[-0.00017017300000000263, -0.00017114699999999...",0.368138
47,nature_dict,IntegralT_2to10,"[15539.175153602439, 15469.948482233931, 15556...",-0.596940


In [22]:
featFull_all = np.vstack(feat_df.loc[[28, 32, 34, 37, 38, 41, 43, 44, 47], 'valueArray']).transpose()

featFull_train  = featFull_all[train_ind]
featFull_test = featFull_all[test_ind]
featFull_secondary_test = featFull_all[secondary_test_ind]

scaler = StandardScaler().fit(featFull_train)
featFull_train_scaled = scaler.transform(featFull_train)
featFull_test_scaled = scaler.transform(featFull_test)
featFull_secondary_test_scaled = scaler.transform(featFull_secondary_test)
featFull_all_scaled = np.concatenate((featFull_train_scaled, featFull_test_scaled, featFull_secondary_test_scaled))

# Train
regEN = ElasticNetCV(l1_ratio=np.linspace(0.01,1,100), cv=4, random_state=20, max_iter=100000, n_jobs=3, tol=1e-2, verbose=0, selection='random').fit(featFull_train_scaled, y_train)
regLN = LinearRegression().fit(featFull_train_scaled, y_train)
model = regEN

benchFullAll = 10**model.predict(featFull_all_scaled)
benchFullTrain = 10**model.predict(featFull_train_scaled)
benchFullTest = 10**model.predict(featFull_test_scaled)
benchFullSecTest = 10**model.predict(featFull_secondary_test_scaled)


print("Benchmark: ElasticNet on Full model",
    "\nMAE_all: %.2f" % (mean_absolute_error(y_true = 10**y_all, y_pred = benchFullAll)),
    "\nMAE_train: %.2f" % (mean_absolute_error(y_true = 10**y_train, y_pred = benchFullTrain)),
    "\nMAE_test: %.2f" % (mean_absolute_error(y_true = 10**y_test, y_pred = benchFullTest)),
    "\nMAE_2ndTest: %.2f" % (mean_absolute_error(y_true = 10**y_secondary_test, y_pred = benchFullSecTest)),
    
    "\nMAPE_all: %.1f %%" % (mean_absolute_percentage_error(y_true = 10**y_all, y_pred = benchFullAll)* 100),
    "\nMAPE_train: %.1f %%" % (mean_absolute_percentage_error(y_true = 10**y_train, y_pred = benchFullTrain)* 100),
    "\nMAPE_test: %.1f %%" % (mean_absolute_percentage_error(y_true = 10**y_test, y_pred = benchFullTest)* 100),
    "\nMAPE_2ndTest: %.1f %%" % (mean_absolute_percentage_error(y_true = 10**y_secondary_test, y_pred = benchFullSecTest)* 100),
    
    "\nRMSE_all: %.2f" % (math.sqrt(mean_squared_error(y_true = 10**y_all, y_pred = benchFullAll))),
    "\nRMSE_train: %.2f" % (math.sqrt(mean_squared_error(y_true = 10**y_train, y_pred = benchFullTrain))),
    "\nRMSE_test: %.2f" % (math.sqrt(mean_squared_error(y_true = 10**y_test, y_pred = benchFullTest))),
    "\nRMSE_2ndTest: %.2f" % (math.sqrt(mean_squared_error(y_true = 10**y_secondary_test, y_pred = benchFullSecTest))),
    
    "\nRMSPE_all: %.1f %%" % (root_mean_squared_percentage_error(y_true = 10**y_all, y_pred = benchFullAll) * 100),
    "\nRMSPE_train: %.1f %%" % (root_mean_squared_percentage_error(y_true = 10**y_train, y_pred = benchFullTrain)* 100),
    "\nRMSPE_test: %.1f %%" % (root_mean_squared_percentage_error(y_true = 10**y_test, y_pred = benchFullTest)* 100),
    "\nRMSPE_2ndTest: %.1f %%" % (root_mean_squared_percentage_error(y_true = 10**y_secondary_test, y_pred = benchFullSecTest)* 100)
    )
print("r2-scores")
print("All data r2-score: %.3f" % (r2_score(y_true = 10**y_all, y_pred = benchFullAll)))
print("Train data r2-score: %.3f" % (r2_score(y_true = 10**y_train, y_pred = benchFullTrain)))
print("Test data r2-score: %.3f" % (r2_score(y_true = 10**y_test, y_pred = benchFullTest)))
print("Secondary test data r2-score: %.3f" % (r2_score(y_true = 10**y_secondary_test, y_pred = benchFullSecTest)))

Benchmark: ElasticNet on Full model 
MAE_all: 166.80 
MAE_train: 79.87 
MAE_test: 122.53 
MAE_2ndTest: 302.38 
MAPE_all: 17.1 % 
MAPE_train: 10.4 % 
MAPE_test: 14.5 % 
MAPE_2ndTest: 26.7 % 
RMSE_all: 259.42 
RMSE_train: 142.72 
RMSE_test: 201.83 
RMSE_2ndTest: 378.54 
RMSPE_all: 21.8 % 
RMSPE_train: 15.3 % 
RMSPE_test: 18.2 % 
RMSPE_2ndTest: 29.6 %
r2-scores
All data r2-score: 0.503
Train data r2-score: 0.805
Test data r2-score: 0.727
Secondary test data r2-score: -0.763


## Plot the bar graph: comparing Temperature (& Hybrid) model with the three Severson benchmarks

In [23]:
textSize= 30
barLabelSize = 17
fontName='Microsoft Sans Serif'
lineWidth = 4
markerSize = 8
dataSource = "Benchmark models"

datasetList = ["SNL-NMC", "SNL-NCA", "SNL-LFP", "UL-NCA", "XJTU", "TRI (Primary)", "TRI (Secondary)"]

width = 0.35  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(1, 1, figsize=(24, 10), dpi=1000)
for axis in ['bottom', 'left']: ax.spines[axis].set_linewidth(lineWidth)
for axis in ['top', 'right']: ax.spines[axis].set_linewidth(0)
    
# Feature plot
ax.tick_params(labelsize=textSize, left=True, bottom=True)
ax.set_axisbelow(False)

# Y-axis
ax.yaxis.set_tick_params(direction='in', length=12, which='major', width=4)
ax.yaxis.set_tick_params(length=-14, which='minor', width=2, labelsize=textSize)
yTickArr = np.arange(0, 101, 20)
yTickMinorArr = np.arange(10, 101, 20)
ax.set_yticks(yTickArr)
ax.set_yticklabels(yTickArr, fontname=fontName, x = -0.015)
ax.set_yticks(yTickMinorArr, minor=True)
ax.set_yticklabels(['-']*len(yTickMinorArr), minor=True)
ax.set_ylim(np.min(yTickArr), np.max(yTickArr)+5)
ax.yaxis.set_label_coords(-0.055, 0.5)
ax.set_ylabel("RMSPE (%)", fontsize=textSize+4, fontname=fontName)

# X-axis
x = np.arange(0, 1.4*len(datasetList), 1.4).astype('float64')
x[-1] = x[-1] + width*2
ax.xaxis.set_tick_params(direction='out', length=12, which='major', width=4)
xTickArr = x + 0.35
xTickArr[-2:] += width/2
ax.set_xticks(xTickArr)
ax.set_xticklabels(datasetList, fontname=fontName, fontsize=textSize, rotation=0, y = -0.025)
ax.set_xlim(np.min(x)-width*1.5, np.max(x)+1.4+width*1.5)

# Plot
modelResults = {
    "This work": [21.8, 7.4, 12.3, 18.4, 6.4, 18.5, 20.9],
    "Variance": [92.9, 19.0, 37.7, 53.4, 29.7, 36.8, 39.2],
    "Discharge": [87.1, 18.6, 38.9, 31.7, 25.2, 35.8, 44.2]
}

colorList = ['steelblue', 'gray', 'gray']
alphaList = [0.6, 0.5, 1.0]

# TRI (Hybrid)
rects = ax.bar(x[-2:]+width, [16.1, 18.3], width, label='This work (Hybrid)', color=colorList[0], alpha=1.0)
ax.bar_label(rects, padding=4, fontsize=barLabelSize, fmt='%.1f')
rects = ax.bar(x[-2:]+4*width, [18.1, 29.7], width, label='Full', color='black', alpha=0.8)
ax.bar_label(rects, padding=4, fontsize=barLabelSize, fmt='%.1f')

# Everything else but TRI
x_copy = x
for counter, (attribute, measurement) in enumerate(modelResults.items()):
    if(counter==1): x_copy[-2:] += width
    offset = width * multiplier
    rects = ax.bar(x_copy + offset, measurement, width, label=attribute, color=colorList[counter], alpha=alphaList[counter])
    ax.bar_label(rects, padding=4, fontsize=barLabelSize, fmt='%.1f')
    multiplier += 1


ax.plot((1), (np.min(yTickArr)), ls="", marker=">", ms=15, color="k", transform=ax.get_yaxis_transform(), clip_on=False)
ax.plot((np.min(x)-width*1.5), (1), ls="", marker="^", ms=15, color="k", transform=ax.get_xaxis_transform(), clip_on=False)


# Initiate save variable for saving directory
# fig.savefig(save, dpi=1000, bbox_inches='tight', facecolor=(1,1,1,0))

fig.show()